In [4]:
import requests
import time
import pandas as pd 
import numpy as np
from datetime import datetime 
import time 
from datetime import timedelta

In [5]:
# URL base de la API CoinGecko
url = 'https://api.coingecko.com/api/v3'

# Obtengo una lista de criptomonedas y sus IDs para despues solicitar a la API los datos historicos 
coins_id_list_url = f'{url}/coins/list'
response = requests.get(coins_id_list_url)

if response.status_code == 200:
    coins_id_list = response.json()
    # Creo una lista de los IDs de las 10 criptomonedas más relevantes
    relevant_crypto_ids = ['bitcoin', 'ethereum', 'cardano', 'binancecoin', 'solana', 'ripple', 'polkadot', 'dogecoin', 'avalanche-2', 'algorand']

    # Filtro las criptos relevantes y obtengo sus IDs
    relevant_ids = [coin['id'] for coin in coins_id_list if coin['id'] in relevant_crypto_ids]
else:
    print(f'Error al obtener la lista de criptomonedas. Código de estado: {response.status_code}')

In [6]:
# Creo un diccionario vacio 
histori_data_dict = {}

# Obtengo los datos históricos para cada criptomoneda
for crypto_id in relevant_ids:
    endpoint = f'/coins/{crypto_id}/market_chart'
    intervalo_dias = 30  # Intervalo de días para cada solicitud
    today = datetime.now()
    params = {'vs_currency': 'usd', 'days': intervalo_dias}
    
    data_list = []

    while today.year > 2021:  # Obtengo datos desde 2022 hasta ahora
        response = requests.get(url + endpoint, params=params)
        
        if response.status_code == 200:
            historical_data = response.json()
            for entry in historical_data['prices']:
                timestamp = entry[0]
                price = entry[1]
                volume = next((vol[1] for vol in historical_data['total_volumes'] if vol[0] == timestamp), None)
                market_cap = next((cap[1] for cap in historical_data['market_caps'] if cap[0] == timestamp), None)
                data_list.append({
                    'Timestamp': datetime.fromtimestamp(timestamp / 1000),
                    'Price': price,
                    'Volume': volume,
                    'MarketCap': market_cap
                })
            # Agrego un retraso de 6 segundos entre las solicitudes para respetar el límite de 10 llamadas por minuto
            time.sleep(6)
        
        else:
            print(f'Error al obtener datos históricos para {crypto_id}. Código de estado: {response.status_code}')
        
        today -= timedelta(days=intervalo_dias)
        params['days'] = intervalo_dias
    
    # Guardo datos obtenidos en el diccionario
    histori_data_dict[crypto_id] = data_list


In [7]:
histori_data_dict

{'algorand': [{'Timestamp': datetime.datetime(2023, 7, 25, 14, 1, 29, 699000),
   'Price': 0.10906712116896111,
   'Volume': 24753965.412301335,
   'MarketCap': 850593564.1774614},
  {'Timestamp': datetime.datetime(2023, 7, 25, 15, 0, 35, 440000),
   'Price': 0.10949099364337353,
   'Volume': 24938730.307127655,
   'MarketCap': 853725326.9708904},
  {'Timestamp': datetime.datetime(2023, 7, 25, 16, 1, 39, 769000),
   'Price': 0.10952297965550326,
   'Volume': 24958141.092670765,
   'MarketCap': 854414906.4673696},
  {'Timestamp': datetime.datetime(2023, 7, 25, 17, 1, 40, 427000),
   'Price': 0.10953219992983804,
   'Volume': 30338442.481692247,
   'MarketCap': 853319308.2337407},
  {'Timestamp': datetime.datetime(2023, 7, 25, 18, 1, 14, 43000),
   'Price': 0.10931909067123136,
   'Volume': 25284192.25267738,
   'MarketCap': 852546540.5815152},
  {'Timestamp': datetime.datetime(2023, 7, 25, 19, 0, 21, 861000),
   'Price': 0.10977625216758202,
   'Volume': 25581716.281644948,
   'MarketCa

In [8]:
# Creo DataFrames individuales por cada moneda
data_frames = {}
for crypto_id, data_list in histori_data_dict.items():
    df = pd.DataFrame(data_list)
    data_frames[crypto_id] = df

In [9]:
# Separo los df por cada moneda
df_algorand = data_frames['algorand']
df_avalanche = data_frames['avalanche-2']
df_dogecoin = data_frames['dogecoin']
df_polkadot = data_frames['polkadot']
df_ripple = data_frames['ripple']
df_solana = data_frames['solana']
df_binancecoin = data_frames['binancecoin']
df_cardano = data_frames['cardano']
df_ethereum = data_frames['ethereum']
df_bitcoin = data_frames['bitcoin']

In [24]:
# Concateno los DataFrames en un único DataFrame
df_criptos = pd.concat(data_frames.values(), keys=data_frames.keys())


In [25]:
# Repito las claves para que coincidan con la cantidad de filas
crypto_ids = [crypto_id for crypto_id in data_frames.keys() for _ in range(len(data_frames[crypto_id]))]

# Agrego la columna 'crypto_id' al DataFrame
df_criptos['crypto_id'] = crypto_ids



In [26]:
# Exporto df para utilizarlo en csv
df_algorand.to_csv('df_algorand.csv', index=False, encoding='437')
df_avalanche.to_csv('df_avalanche.csv', index=False, encoding='437')
df_dogecoin.to_csv('df_dogecoin.csv', index=False, encoding='437')
df_polkadot.to_csv('df_polkadot.csv', index=False, encoding='437')
df_ripple.to_csv('df_ripple.csv', index=False, encoding='437')
df_solana.to_csv('df_solana.csv', index=False, encoding='437')
df_binancecoin.to_csv('df_binancecoin.csv', index=False, encoding='437')
df_cardano.to_csv('df_cardano.csv', index=False, encoding='437')
df_ethereum.to_csv('df_ethereum.csv', index=False, encoding='437')
df_bitcoin.to_csv('df_bitcoin.csv', index=False, encoding='437')
df_criptos.to_csv('df_criptos.csv', index=False, encoding='437')